### Trabalho 2 - Conway's Game of Life
###### Grupo 19

Tiago Passos Rodrigues - A96414

### Implementação

Definir o inputs do $N$ tamanho do campo de jogo, vetores constantes das bordas com $prob$ (probabilidade) das bordas serem 1, caso contrário 0.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT
import random, numpy

N = 5 #tamanho da grelha

#centro = (random.randint(0,N),random.randint(0,N)) #random menos das bordas
centro = (2,2)
print(centro)

# probabilidade para o valor de 1 nas bordas
prob = 0.5

# criacao de 2 vetores costantes e uma constante da ponta da borda
zero_zero = numpy.random.choice([0,1], p=[1 - prob, prob])
vetorx = []
for i in range(N):
    vetorx.append(numpy.random.choice([0,1], p=[1 - prob, prob]))
vetory = []
for i in range(N):
    vetory.append(numpy.random.choice([0,1], p=[1 - prob, prob]))
    
print("Valor do 0_0 = ", zero_zero)
print("Borda pelo linha do x = ", vetorx)
print("Borda pela linha do y = ", vetory)
# condicoes para os simbolos da grelha que estejam na ponta

(2, 2)
Valor do 0_0 =  1
Borda pelo linha do x =  [1, 0, 0, 1, 1]
Borda pela linha do y =  [0, 0, 1, 0, 1]


### Funções auxiliares

Criação das funções para as células, contagem da vizinhança, estado de inicio e transições de grelha para grelha.

In [2]:
# criacao do campo de jogo
def declare(i,j,k):
    return Symbol('i'+str(i)+'j'+str(j)+'_'+str(k),INT)

# devolve o nº de vizinhos a 1
def vizinhanca(celula,grelha):
    x = celula[0]
    y = celula[1]

    if y == N-1 and x == N-1:
        return [grelha[x-1][y], grelha[x][y-1], grelha[x-1][y-1]]
    elif y == N-1:
        return [grelha[x+1][y], grelha[x-1][y], grelha[x][y-1], grelha[x+1][y-1], grelha[x-1][y-1]]
    elif x == N-1:
        return [grelha[x-1][y] , grelha[x][y-1] , grelha[x][y+1] 
            , grelha[x-1][y+1], grelha[x-1][y-1]]
    elif x == 0 and y == 0:
        return [zero_zero, vetorx[0], vetorx[1], vetory[0], vetory[1], grelha[x+1][y], grelha[x][y+1], grelha[x+1][y+1]]
    elif x == 0:
        return [vetory[y-1],vetory[y],vetory[y+1],grelha[x][y+1],grelha[x+1][y+1],grelha[x+1][y],grelha[x+1][y-1],grelha[x][y-1]]
    elif y == 0:
        return [vetorx[x-1],vetory[y],vetory[y+1],grelha[x+1][y],grelha[x+1][y+1],grelha[x][y+1],grelha[x-1][y+1],grelha[x-1][y]]
    else:
        return [grelha[x+1][y], grelha[x-1][y], grelha[x][y-1], grelha[x][y+1] 
            ,grelha[x+1][y+1], grelha[x-1][y+1],grelha[x+1][y-1], grelha[x-1][y-1]]

#Estado inicial do jogo, 3 X 3 tudo a 1
def init(grelha,s):
    x = centro[0]
    y = centro[1]
    grelhas_not_zero = [grelha[x][y],grelha[x+1][y],grelha[x-1][y],grelha[x][y-1],grelha[x][y+1],grelha[x+1][y+1],
                       grelha[x-1][y+1],grelha[x+1][y-1],grelha[x-1][y-1]]
    for i in range(N):
        for j in range(N):
            if grelha[i][j] not in grelhas_not_zero:
                s.add_assertion(Equals(grelha[i][j],Int(0)))
    return And(Equals(grelha[x][y],Int(1)),
                Equals(grelha[x+1][y],Int(1)),
                Equals(grelha[x-1][y],Int(1)),
                Equals(grelha[x][y-1],Int(1)),
                Equals(grelha[x][y+1],Int(1)),
                Equals(grelha[x+1][y+1],Int(1)),
                Equals(grelha[x-1][y+1],Int(1)),
                Equals(grelha[x+1][y-1],Int(1)),
                Equals(grelha[x-1][y-1],Int(1)))


# transições grelha atual de uma célula para a próxima grelha
def trans(celula,prox,par,curr):
    (i,j) = par
    t0 = And(Equals(celula,Int(0)),
            Equals(sum(vizinhanca(par,curr)), Int(3)),
            Equals(prox,Int(1))
        )

    t1 = And(Equals(celula,Int(1)),
                Or(Equals(sum(vizinhanca(par,curr)), Int(3)), Equals(sum(vizinhanca(par,curr)), Int(2))),
                Equals(prox,Int(1))
            )

    t2 = And(Equals(celula,Int(1)),
                Not(Or(Equals(sum(vizinhanca(par,curr)), Int(3)), Equals(sum(vizinhanca(par,curr)), Int(2)))),
                Equals(prox,Int(0))
            )

    t3 = And(Equals(celula,Int(0)),
                Not(Equals(sum(vizinhanca(par,curr)), Int(3))),
                Equals(prox,Int(0))
            )
    return Or(t0,t1,t2,t3)
    
def alguem_vivo(grelha):
    lista = []
    for i in range(N):
        for j in range(N):
            lista.append(grelha[i][j])
    return GT(sum(lista),Int(1))

def esteve_vivo(grelhas,celula):
    (x,y) = celula
    lista = []
    for grelha in grelhas:
        lista.append(grelha[x][y])
    return GT(sum(lista),Int(1))
        
    

### Jogo

Declaração das grelhas do jogo, estado do início e transições.

In [3]:
with Solver(name="z3") as s:
    
    grelhas = []
    for k in range(N):
        grelhas.append([[declare(i,j,k) for j in range(N)] for i in range(N)])    
    
    # tem de ter valores 0 ou 1
    for grelha in grelhas:
        for i in range(N):
            for j in range(N):
                s.add_assertion(Or(Equals(grelha[i][j],Int(1)), Equals(grelha[i][j],Int(0))))

    
    # estado inicial da grelha
    s.add_assertion(init(grelhas[0],s))
    
    # transicoes das celulas
    for k in range(N-1):
        for i in range(N):
            for j in range(N):
                s.add_assertion(trans(grelhas[k][i][j], grelhas[k+1][i][j],(i,j),grelhas[k]))
    
    if s.solve():
        print("\nInicio: \n")
        for grelha in grelhas:
            #print(s.get_value(grelhas[g][0][0]))
            for i in range(N):
                if i == 0:
                    print(zero_zero, end = "   ")
                    for j in range(N):
                        print(vetorx[j], end = " ")
                    print()
                    print("------------------------")
                print(vetory[i], end = " | ")
                for j in range(N):
                    #print(grelhas[g][i][j], end = " ")
                    print(s.get_value(grelha[i][j]) , end = " ")
                print()
            print("\ntransicao\n")


Inicio: 

1   1 0 0 1 1 
------------------------
0 | 0 0 0 0 0 
0 | 0 1 1 1 0 
1 | 0 1 1 1 0 
0 | 0 1 1 1 0 
1 | 0 0 0 0 0 

transicao

1   1 0 0 1 1 
------------------------
0 | 1 1 0 0 0 
0 | 1 1 0 1 0 
1 | 1 0 0 0 1 
0 | 0 1 0 1 0 
1 | 0 0 1 0 0 

transicao

1   1 0 0 1 1 
------------------------
0 | 0 0 0 1 0 
0 | 0 0 1 0 0 
1 | 1 0 0 1 1 
0 | 0 1 1 1 0 
1 | 0 0 1 0 0 

transicao

1   1 0 0 1 1 
------------------------
0 | 0 0 1 1 0 
0 | 0 0 1 0 1 
1 | 0 0 0 0 1 
0 | 0 1 0 0 1 
1 | 0 1 1 1 0 

transicao

1   1 0 0 1 1 
------------------------
0 | 0 1 1 0 1 
0 | 0 0 1 0 1 
1 | 0 0 0 0 1 
0 | 0 1 0 0 1 
1 | 1 1 1 1 0 

transicao



### Invariantes

1. Todos os estados acessíveis contém pelo menos uma célula viva.

In [4]:
with Solver(name="z3") as s:
    
    grelhas = []
    for k in range(N):
        grelhas.append([[declare(i,j,k) for j in range(N)] for i in range(N)])    

    for grelha in grelhas:
        for i in range(N):
            for j in range(N):
                s.add_assertion(Or(Equals(grelha[i][j],Int(1)), Equals(grelha[i][j],Int(0))))

    s.add_assertion(init(grelhas[0],s))

    for k in range(N-1):
        for i in range(N):
            for j in range(N):
                s.add_assertion(trans(grelhas[k][i][j], grelhas[k+1][i][j],(i,j),grelhas[k]))
                
    # adicionar o invariante
    for grelha in grelhas:
        s.add_assertion(Not(And(alguem_vivo(grelha))))
    
    if s.solve():
        print("Invariante não válido")
    else:
        print("Invariante válido")

Invariante válido


2. Todos os estados acessíveis contém pelo menos uma célula viva.

In [5]:
with Solver(name="z3") as s:
    
    grelhas = []
    for k in range(N):
        grelhas.append([[declare(i,j,k) for j in range(N)] for i in range(N)])    

    for grelha in grelhas:
        for i in range(N):
            for j in range(N):
                s.add_assertion(Or(Equals(grelha[i][j],Int(1)), Equals(grelha[i][j],Int(0))))

    s.add_assertion(init(grelhas[0],s))

    for k in range(N-1):
        for i in range(N):
            for j in range(N):
                s.add_assertion(trans(grelhas[k][i][j], grelhas[k+1][i][j],(i,j),grelhas[k]))
                
    # adicionar o invariante
    s.add_assertion(Not(And(esteve_vivo(grelhas,(i,j)) for i in range(N) for j in range(N))))
    
    if s.solve():
        print("Invariante não válido")
    else:
        print("Invariante válido")

Invariante não válido
